In [35]:
import pandas as pd
train=pd.read_csv("C:/Users/user/Desktop/open/train.csv")
test=pd.read_csv("C:/Users/user/Desktop/open/test.csv")

In [36]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7497 non-null   object 
 1   제조사       7497 non-null   object 
 2   모델        7497 non-null   object 
 3   차량상태      7497 non-null   object 
 4   배터리용량     4786 non-null   float64
 5   구동방식      7497 non-null   object 
 6   주행거리(km)  7497 non-null   int64  
 7   보증기간(년)   7497 non-null   int64  
 8   사고이력      7497 non-null   object 
 9   연식(년)     7497 non-null   int64  
 10  가격(백만원)   7497 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 644.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        846 non-null    object 
 1   제조사       846 non-null    object 
 2   모델        846 non-null    object 
 3   차량상태      846 non

In [37]:
train1=train.copy()
test1=test.copy()

In [38]:
train1

,ID,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원)
0,TRAIN_0000,P사,TayGTS,Nearly New,86.077,AWD,13642,0,No,2,159.66
1,TRAIN_0001,K사,Niro,Nearly New,56.000,FWD,10199,6,No,0,28.01
2,TRAIN_0002,A사,eT,Brand New,91.200,AWD,2361,7,No,0,66.27
3,TRAIN_0003,A사,RSeTGT,Nearly New,NaN,AWD,21683,3,No,0,99.16
4,TRAIN_0004,B사,i5,Pre-Owned,61.018,AWD,178205,1,No,0,62.02
...,...,...,...,...,...,...,...,...,...,...,...
7492,TRAIN_7492,H사,ION5,Brand New,NaN,AWD,3773,10,No,0,35.95
7493,TRAIN_7493,B사,i3,Pre-Owned,46.000,RWD,135411,2,No,0,23.40
7494,TRAIN_7494,P사,TayCT,Brand New,NaN,AWD,1363,2,No,0,120.00
7495,TRAIN_7495,B사,i3,Nearly New,56.000,RWD,39445,6,No,2,24.00


In [39]:
def create_improved_features(data):
    
    data['제조사_모델_상태'] = data['제조사'] + '_' + data['모델']+ '_' + data['차량상태']
    
    data['주행거리비율'] = data.apply(
        lambda row: row['주행거리(km)'] if row['연식(년)'] == 0 else row['주행거리(km)'] / row['연식(년)'],
        axis=1)
    
    current_year = 2025 
    data['사용연한주행거리비율'] = data['주행거리(km)'] / (current_year - data['연식(년)'] + 1)

    data['전비(km/kWh)'] = data['주행거리(km)'] / data['배터리용량']
    
    data['중고여부'] = data['차량상태'].apply(lambda x: 0 if x == 'Brand New' else 1)
    
    drive_weight = {'AWD': 1.2, 'RWD': 1.1, 'FWD': 1.0}
    data['구동방식가중치'] = data['구동방식'].map(drive_weight)
    
    data['사고가중치'] = data['사고이력'].apply(lambda x: 1 if x == 'Yes' else 0)
    
    return data

# train_data와 test_data에 적용
train1 = create_improved_features(train1)
test1 = create_improved_features(test1)

# 생성된 데이터 확인
print(train1[['제조사_모델_상태','주행거리비율','사용연한주행거리비율', '전비(km/kWh)', '중고여부','구동방식가중치', '사고가중치']])
print(test1[['제조사_모델_상태','주행거리비율','사용연한주행거리비율', '전비(km/kWh)', '중고여부','구동방식가중치', '사고가중치']])


                 제조사_모델_상태    주행거리비율  사용연한주행거리비율   전비(km/kWh)  중고여부  구동방식가중치  \
0     P사_TayGTS_Nearly New    6821.0    6.740119   158.486007     1      1.2   
1       K사_Niro_Nearly New   10199.0    5.034057   182.125000     1      1.0   
2          A사_eT_Brand New    2361.0    1.165350    25.888158     0      1.2   
3     A사_RSeTGT_Nearly New   21683.0   10.702369          NaN     1      1.2   
4          B사_i5_Pre-Owned  178205.0   87.959033  2920.531646     1      1.2   
...                    ...       ...         ...          ...   ...      ...   
7492     H사_ION5_Brand New    3773.0    1.862290          NaN     0      1.2   
7493       B사_i3_Pre-Owned  135411.0   66.836624  2943.717391     1      1.1   
7494    P사_TayCT_Brand New    1363.0    0.672754          NaN     0      1.2   
7495      B사_i3_Nearly New   19722.5   19.488636   704.375000     1      1.1   
7496       T사_MY_Pre-Owned   80215.0   39.592794  1544.378129     1      1.2   

      사고가중치  
0         0  
1         0

In [40]:
import pandas as pd
import numpy as np
from scipy import stats

features = ['제조사', '모델', '차량상태','제조사_모델_상태', '구동방식', '사고이력']
target = '배터리용량'

original_train1 = train1.copy()

if train1[target].isnull().any():
    train1 = train1.dropna(subset=[target])

anova_results = {}
for feature in features:
    if feature in train1.columns:
        group_data = [train1[target][train1[feature] == category] for category in train1[feature].unique()]
        f_stat, p_value = stats.f_oneway(*group_data)
        anova_results[feature] = (f_stat, p_value)

print("ANOVA 결과 (F-statistic, p-value):")
for feature, (f_stat, p_value) in anova_results.items():
    print(f"{feature}: F-statistic = {f_stat}, p-value = {p_value}")

significant_features = [feature for feature, (f_stat, p_value) in anova_results.items() if p_value < 0.05]
print("유의미한 변수들:", significant_features)

train1 = original_train1.copy()


ANOVA 결과 (F-statistic, p-value):
제조사: F-statistic = 284.7074906220911, p-value = 1.43179867482e-312
모델: F-statistic = 341.0160771230864, p-value = 0.0
차량상태: F-statistic = 5936.351849621947, p-value = 0.0
제조사_모델_상태: F-statistic = 2564.800512594081, p-value = 0.0
구동방식: F-statistic = 230.34414840207486, p-value = 3.1037841474407706e-96
사고이력: F-statistic = 0.42380557755032844, p-value = 0.515075483805737
유의미한 변수들: ['제조사', '모델', '차량상태', '제조사_모델_상태', '구동방식']


In [41]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

features = ['보증기간(년)', '연식(년)', '주행거리(km)','주행거리비율','사용연한주행거리비율', '전비(km/kWh)', '중고여부','구동방식가중치', '사고가중치']
target = '배터리용량'

original_train1 = train1.copy()

train1_clean = train1.dropna(subset=[target] + features)

results = {}
for feature in features:
    corr, p_value = pearsonr(train1_clean[feature], train1_clean[target])
    results[feature] = {'correlation': corr, 'p-value': p_value}

for feature, values in results.items():
    print(f"{feature}: 상관계수 = {values['correlation']:.4f}, p-value = {values['p-value']:.4f}")

significant_results = [feature for feature, values in results.items() if values['p-value'] < 0.05]
print("유의미한 변수들:", significant_results)

train1 = original_train1.copy()


보증기간(년): 상관계수 = 0.5582, p-value = 0.0000
연식(년): 상관계수 = -0.0218, p-value = 0.1308
주행거리(km): 상관계수 = -0.5765, p-value = 0.0000
주행거리비율: 상관계수 = -0.5677, p-value = 0.0000
사용연한주행거리비율: 상관계수 = -0.5766, p-value = 0.0000
전비(km/kWh): 상관계수 = -0.6515, p-value = 0.0000
중고여부: 상관계수 = -0.8152, p-value = 0.0000
구동방식가중치: 상관계수 = 0.2820, p-value = 0.0000
사고가중치: 상관계수 = -0.0094, p-value = 0.5151
유의미한 변수들: ['보증기간(년)', '주행거리(km)', '주행거리비율', '사용연한주행거리비율', '전비(km/kWh)', '중고여부', '구동방식가중치']


In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

features = ['제조사', '모델', '차량상태', '구동방식', '제조사_모델_상태', '주행거리(km)', '보증기간(년)',
            '주행거리비율', '중고여부', '구동방식가중치']
target = '배터리용량'

data_train = train1[train1[target].notnull()]
data_missing = train1[train1[target].isnull()]

X_train = data_train[features]
y_train = data_train[target]
X_missing = data_missing[features]

categorical_cols = ['제조사', '모델', '차량상태', '구동방식', '제조사_모델_상태']
numerical_cols = ['주행거리(km)', '보증기간(년)', '주행거리비율', '중고여부', '구동방식가중치']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),  # 결측치 평균 대체
            ('scaler', StandardScaler())  # 스케일링
        ]), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # 원-핫 인코딩
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

model.fit(X_train, y_train)

predicted_values = model.predict(X_missing)

train1.loc[train1[target].isnull(), target] = predicted_values

train1['사용연한주행거리비율'] = train1['주행거리(km)'] / (train1['배터리용량'])
train1['전비(km/kWh)'] = train1['주행거리(km)'] / (train1['배터리용량'])

print(train1.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          7497 non-null   object 
 1   제조사         7497 non-null   object 
 2   모델          7497 non-null   object 
 3   차량상태        7497 non-null   object 
 4   배터리용량       7497 non-null   float64
 5   구동방식        7497 non-null   object 
 6   주행거리(km)    7497 non-null   int64  
 7   보증기간(년)     7497 non-null   int64  
 8   사고이력        7497 non-null   object 
 9   연식(년)       7497 non-null   int64  
 10  가격(백만원)     7497 non-null   float64
 11  제조사_모델_상태   7497 non-null   object 
 12  주행거리비율      7497 non-null   float64
 13  사용연한주행거리비율  7497 non-null   float64
 14  전비(km/kWh)  7497 non-null   float64
 15  중고여부        7497 non-null   int64  
 16  구동방식가중치     7497 non-null   float64
 17  사고가중치       7497 non-null   int64  
dtypes: float64(6), int64(5), object(7)
memory usage: 1.0+ MB
None


In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

features = ['제조사', '모델', '차량상태', '구동방식', '제조사_모델_상태', '주행거리(km)', '보증기간(년)',
            '주행거리비율', '중고여부', '구동방식가중치']
target = '배터리용량'

data_train = test1[test1[target].notnull()]
data_missing = test1[test1[target].isnull()]

X_train = data_train[features]
y_train = data_train[target]
X_missing = data_missing[features]

categorical_cols = ['제조사', '모델', '차량상태', '구동방식', '제조사_모델_상태']
numerical_cols = ['주행거리(km)', '보증기간(년)', '주행거리비율', '중고여부', '구동방식가중치']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[  
            ('scaler', StandardScaler()) 
        ]), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # 원-핫 인코딩
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

model.fit(X_train, y_train)

predicted_values = model.predict(X_missing)

test1.loc[test1[target].isnull(), target] = predicted_values

test1['사용연한주행거리비율'] = test1['주행거리(km)'] / (test1['배터리용량'])
test1['전비(km/kWh)'] = test1['주행거리(km)'] / (test1['배터리용량']) 

print(test1.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          846 non-null    object 
 1   제조사         846 non-null    object 
 2   모델          846 non-null    object 
 3   차량상태        846 non-null    object 
 4   배터리용량       846 non-null    float64
 5   구동방식        846 non-null    object 
 6   주행거리(km)    846 non-null    int64  
 7   보증기간(년)     846 non-null    int64  
 8   사고이력        846 non-null    object 
 9   연식(년)       846 non-null    int64  
 10  제조사_모델_상태   846 non-null    object 
 11  주행거리비율      846 non-null    float64
 12  사용연한주행거리비율  846 non-null    float64
 13  전비(km/kWh)  846 non-null    float64
 14  중고여부        846 non-null    int64  
 15  구동방식가중치     846 non-null    float64
 16  사고가중치       846 non-null    int64  
dtypes: float64(5), int64(5), object(7)
memory usage: 112.5+ KB
None


In [44]:
import pandas as pd
import numpy as np
from scipy import stats

features = ['제조사', '모델', '차량상태','제조사_모델_상태', '구동방식', '사고이력']
target = '가격(백만원)'

original_train1 = train1.copy()

if train1[target].isnull().any():
    train1 = train1.dropna(subset=[target])

anova_results = {}
for feature in features:
    if feature in train1.columns:
        group_data = [train1[target][train1[feature] == category] for category in train1[feature].unique()]
        f_stat, p_value = stats.f_oneway(*group_data)
        anova_results[feature] = (f_stat, p_value)

print("ANOVA 결과 (F-statistic, p-value):")
for feature, (f_stat, p_value) in anova_results.items():
    print(f"{feature}: F-statistic = {f_stat}, p-value = {p_value}")

significant_features = [feature for feature, (f_stat, p_value) in anova_results.items() if p_value < 0.05]
print("유의미한 변수들:", significant_features)

train1 = original_train1.copy()


ANOVA 결과 (F-statistic, p-value):
제조사: F-statistic = 5505.294820901025, p-value = 0.0
모델: F-statistic = 29840.133235899004, p-value = 0.0
차량상태: F-statistic = 17.825953441308222, p-value = 1.8907920861433577e-08
제조사_모델_상태: F-statistic = 33141.78950854622, p-value = 0.0
구동방식: F-statistic = 962.9997426963447, p-value = 0.0
사고이력: F-statistic = 0.16045499032938393, p-value = 0.6887493806620659
유의미한 변수들: ['제조사', '모델', '차량상태', '제조사_모델_상태', '구동방식']


In [45]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

features = ['보증기간(년)', '연식(년)','배터리용량','주행거리(km)','주행거리비율','사용연한주행거리비율', '전비(km/kWh)', '중고여부','구동방식가중치', '사고가중치']
target = '가격(백만원)'

original_train1 = train1.copy()

train1_clean = train1.dropna(subset=[target] + features)

results = {}
for feature in features:
    corr, p_value = pearsonr(train1_clean[feature], train1_clean[target])
    results[feature] = {'correlation': corr, 'p-value': p_value}

for feature, values in results.items():
    print(f"{feature}: 상관계수 = {values['correlation']:.4f}, p-value = {values['p-value']:.4f}")

significant_results = [feature for feature, values in results.items() if values['p-value'] < 0.05]
print("유의미한 변수들:", significant_results)

train1 = original_train1.copy()


보증기간(년): 상관계수 = -0.3500, p-value = 0.0000
연식(년): 상관계수 = -0.0585, p-value = 0.0000
배터리용량: 상관계수 = 0.3132, p-value = 0.0000
주행거리(km): 상관계수 = -0.0355, p-value = 0.0021
주행거리비율: 상관계수 = -0.0307, p-value = 0.0079
사용연한주행거리비율: 상관계수 = -0.1139, p-value = 0.0000
전비(km/kWh): 상관계수 = -0.1139, p-value = 0.0000
중고여부: 상관계수 = -0.0683, p-value = 0.0000
구동방식가중치: 상관계수 = 0.4503, p-value = 0.0000
사고가중치: 상관계수 = 0.0046, p-value = 0.6887
유의미한 변수들: ['보증기간(년)', '연식(년)', '배터리용량', '주행거리(km)', '주행거리비율', '사용연한주행거리비율', '전비(km/kWh)', '중고여부', '구동방식가중치']


In [46]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

features = ['제조사', '모델', '차량상태', '구동방식','제조사_모델_상태', 
            '보증기간(년)', '연식(년)', '배터리용량', '주행거리(km)', '주행거리비율',
            '사용연한주행거리비율', '전비(km/kWh)', '중고여부', '구동방식가중치']
target = '가격(백만원)'

X_train = train1[features]
y_train = train1[target]

X_test = test1[features]

categorical_cols = ['제조사', '모델', '차량상태', '구동방식','제조사_모델_상태','보증기간(년)', '연식(년)']
numerical_cols = ['배터리용량', '주행거리(km)','주행거리비율', '사용연한주행거리비율', '전비(km/kWh)', '중고여부', '구동방식가중치']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', StandardScaler()) 
        ]), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(objective='reg:squarederror', random_state=42))
])

model.fit(X_train, y_train)

predicted_prices = model.predict(X_test)

test1[target] = predicted_prices

result = test1[['ID', '가격(백만원)']]
print(result)

result.to_csv('C:/Users/user/Desktop/open/sample_submission.csv', index=False)


           ID     가격(백만원)
0    TEST_000  130.453903
1    TEST_001   80.174011
2    TEST_002   65.186852
3    TEST_003   34.889904
4    TEST_004   47.659492
..        ...         ...
841  TEST_841  151.214050
842  TEST_842   38.703236
843  TEST_843   38.888577
844  TEST_844   59.241726
845  TEST_845   22.448332

[846 rows x 2 columns]
